In [ ]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Parâmetros do material (exemplo para Alumina)
material_properties = {
    'Alumina': {
        'rho': 3960,          # Densidade (kg/m³)
        'cp': 880,            # Calor específico (J/(kg·K))
        'k': 30,              # Condutividade térmica (W/(m·K))
        'E': 370e9,           # Módulo de Young (Pa)
        'nu': 0.22,           # Coeficiente de Poisson
        'alpha': 8.5e-6       # Coeficiente de expansão térmica (/K)
    },
    # Adicionar outros materiais conforme necessário
}


In [ ]:
# Escolher o material para simulação
material_name = 'Alumina'
props = material_properties[material_name]


In [ ]:
# Dimensões da peça
length = 0.01       # Comprimento (m)
thickness = 0.001   # Espessura (m)

# Malha
nx = 50
ny = 10
mesh = RectangleMesh(Point(0, 0), Point(length, thickness), nx, ny)

# Espaço de função para temperatura
V = FunctionSpace(mesh, 'P', 1)


In [ ]:
# Condições iniciais e de contorno
T0 = Constant(300)   # Temperatura inicial (K)
T_hot = Constant(623)  # Temperatura na face aquecida (K)

# Definir subdomínios para condições de contorno
boundary_markers = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
boundary_markers.set_all(0)

class BottomBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0.0) and on_boundary

class TopBoundary(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], thickness) and on_boundary

bottom_boundary = BottomBoundary()
top_boundary = TopBoundary()

# Marcar as fronteiras
bottom_boundary.mark(boundary_markers, 1)
top_boundary.mark(boundary_markers, 2)

ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)

# Definir condições de contorno
bc_bottom = DirichletBC(V, T_hot, bottom_boundary)
bc_top = DirichletBC(V, T0, top_boundary)
bcs = [bc_bottom, bc_top]


In [ ]:
# Funções e variáveis
u = Function(V)
u_n = interpolate(T0, V)  # Solução no passo de tempo anterior
v = TestFunction(V)

# Parâmetros de tempo
dt = 0.01                # Passo de tempo (s)
t_end = 1.0              # Tempo total de simulação (s)
num_steps = int(t_end / dt)

# Propriedades do material
rho = props['rho']
cp = props['cp']
k = props['k']

# Definir o problema variacional
F = rho * cp * (u - u_n) / dt * v * dx + k * dot(grad(u), grad(v)) * dx
a, L = lhs(F), rhs(F)


In [ ]:
# Loop de tempo
t = 0
for n in range(num_steps):
    t += dt
    # Resolver
    solve(a == L, u, bcs)
    # Atualizar a solução anterior
    u_n.assign(u)


In [ ]:
# Visualização final da temperatura
plt.figure()
p = plot(u)
plt.colorbar(p)
plt.title('Temperatura em t = {:.2f} s'.format(t_end))
plt.xlabel('Comprimento (m)')
plt.ylabel('Espessura (m)')
plt.show()


In [ ]:
# Análise de tensões térmicas

# Espaço de função para deslocamentos
V_u = VectorFunctionSpace(mesh, 'P', 1)

# Parâmetros mecânicos
E = props['E']
nu = props['nu']
alpha = props['alpha']
lambda_ = E * nu / ((1 + nu) * (1 - 2 * nu))
mu = E / (2 * (1 + nu))

# Definir a forma variacional para elasticidade
def epsilon(u):
    return sym(grad(u))

def sigma(u_disp, T):
    return lambda_ * tr(epsilon(u_disp)) * Identity(2) + 2 * mu * epsilon(u_disp) - \
           lambda_ * alpha * (u - T0) * Identity(2) - 2 * mu * alpha * (u - T0) * Identity(2)

u_disp = TrialFunction(V_u)
v_disp = TestFunction(V_u)

a_elastic = inner(sigma(u_disp, u), epsilon(v_disp)) * dx
L_elastic = dot(Constant((0, 0)), v_disp) * dx


In [ ]:
# Aplicar condições de contorno mecânicas (fixar deslocamentos em uma borda)
def left_boundary(x, on_boundary):
    return near(x[0], 0.0) and on_boundary

bc_disp = DirichletBC(V_u, Constant((0, 0)), left_boundary)

# Resolver o problema elástico
u_sol = Function(V_u)
solve(a_elastic == L_elastic, u_sol, bc_disp)


In [ ]:
# Calcular tensões
stress = sigma(u_sol, u)
V_s = TensorFunctionSpace(mesh, 'P', 1)
stress_proj = project(stress, V_s)

# Visualização das tensões (por exemplo, tensão de von Mises)
from ufl import sqrt, inner
s = stress_proj - (1./3) * tr(stress_proj) * Identity(2)  # Tensor desviador
von_Mises = sqrt(3./2 * inner(s, s))
V_mises = FunctionSpace(mesh, 'P', 1)
von_Mises_proj = project(von_Mises, V_mises)

plt.figure()
p = plot(von_Mises_proj)
plt.colorbar(p)
plt.title('Tensão de von Mises (Pa)')
plt.xlabel('Comprimento (m)')
plt.ylabel('Espessura (m)')
plt.show()
